In [69]:
# from os import chdir
# from google.colab import drive

# drive.mount("/content/drive", force_remount=True)
# chdir("/content/drive/MyDrive/Eliott/files/")

In [1]:
# IMPORTS

import numpy as np
import pandas as pd
import preprocessing as pp
import json


from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from imblearn.over_sampling import SMOTE

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline


%load_ext autoreload
%autoreload 2


2022-03-31 19:21:44.429057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-31 19:21:44.429077: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
file_name_train = 'Corona_NLP_train.csv'
file_name_test = 'Corona_NLP_test.csv'
X_train, y_train = pp.prepare_dataframe(file_name_train,lemmatising=False)
X_test, y_test = pp.prepare_dataframe(file_name_test,lemmatising=False)

On enlève : 
- Les URLS
- Hashtags
- Mentions
- Mots réservés
- Emojis et smileys

- 35525 mots sans lemmatisation
- 30794 avec lemmatisation

# Recherche d'Hyperparamètres

In [3]:
pipeline_sgd = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', SGDClassifier()),
])

pipeline_gb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', GradientBoostingClassifier()),
])

pipeline_rf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier()),
])

pipeline_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression(max_iter=1000)),
])

pipeline_per = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', Perceptron()),
])

pipeline_svc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC(max_iter=10000)),
])


In [4]:
models_and_params = {
    "SGD" : { "model" : pipeline_sgd,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-7, 1e-4, 20),  
                        }
    },
    "GB" : { "model" : pipeline_gb,
              "params" : {
                    "clf__learning_rate": [0.01, 0.1, 0.2],
                    "clf__n_estimators":[10,50,100,200,400,800]
                        }
    },
    "RF" : { "model" : pipeline_rf,
              "params" : {
                    'clf__n_estimators': [10, 100],   
                    'clf__max_depth': [10 ,150,300,600],
                    'clf__min_samples_leaf': [1, 2, 3],   
                    'clf__min_samples_split': [4, 8, 16, 32],
                    'clf__max_features': ['log2', 'sqrt'],
                    'clf__criterion': ['gini', 'entropy'],
                    'clf__warm_start': [True, False] 
                        }
    },
    "LR" : { "model" : pipeline_lr,
              "params" : {
                    'clf__C': [100, 80, 40,20,10, 1.0],
                    'clf__tol': np.linspace(1e-8,1e-4,15)
                        }
    },
    "PER" : { "model" : pipeline_per,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__alpha': np.linspace(1e-8, 1e-4, 20),
                        }
    },
    "SVC" : { "model" : pipeline_svc,
              "params" : {
                    'clf__penalty': ['l2', 'l1', 'elasticnet'],
                    'clf__loss': ['hinge', 'squared_hinge'],
                    'clf__dual' : [False,True]
                        }
    }
    

}

In [5]:
def grid_Search(model_name,subset=-1):

    model = models_and_params[model_name]["model"]
    parameters = models_and_params[model_name]["params"]

    grid_clf = GridSearchCV(model, parameters,  scoring='accuracy', verbose=1 ,n_jobs=-1)
    
    if subset==-1:
        grid_clf.fit(X_train, y_train)
    else:
        grid_clf.fit(X_train[:subset], y_train[:subset])


    print("Best Score: ", grid_clf.best_score_)
    print("Best Params: ", grid_clf.best_params_)

    return grid_clf


### Saving Hyperparameters in JSON

We will first load old weights and updates only if needed. Then we can use the model with best params in the end.

In [6]:
with open('data.json') as json_file:
    dico = json.load(json_file)

### SGD

In [7]:
%%time
grid_sgd = grid_Search("SGD") 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.877785107829688
Best Params:  {'clf__alpha': 1.0615789473684212e-05, 'clf__penalty': 'l1'}
CPU times: user 8.1 s, sys: 6.88 s, total: 15 s
Wall time: 47.8 s


In [8]:
dico['SGD'] = grid_sgd.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Gradient boosting

In [17]:
%%time
grid_gb = grid_Search("GB") 

Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best Score:  0.8184027643721461
Best Params:  {'clf__learning_rate': 0.2, 'clf__n_estimators': 800}
CPU times: user 9min 8s, sys: 1.27 s, total: 9min 10s
Wall time: 40min 15s


In [18]:
dico['GB'] = grid_gb.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Random Forest

In [19]:
%%time
grid_rf = grid_Search("RF") 

Fitting 5 folds for each of 768 candidates, totalling 3840 fits
Best Score:  0.6937577179543217
Best Params:  {'clf__criterion': 'gini', 'clf__max_depth': 600, 'clf__max_features': 'sqrt', 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 32, 'clf__n_estimators': 100, 'clf__warm_start': False}
CPU times: user 1min 26s, sys: 49 s, total: 2min 15s
Wall time: 34min 47s


In [20]:
dico['RF'] = grid_rf.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)


### Logistic Regression

In [21]:
%%time
grid_lr = grid_Search("LR") 

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Best Score:  0.8230920602963577
Best Params:  {'clf__C': 10, 'clf__tol': 1e-08}
CPU times: user 2min 10s, sys: 5min 41s, total: 7min 51s
Wall time: 30min 21s


In [22]:
dico['LR'] = grid_lr.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Perceptron

In [23]:
%%time
grid_per = grid_Search("PER") 

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Score:  0.8128628567555942
Best Params:  {'clf__alpha': 5.272631578947369e-06, 'clf__penalty': 'l1'}
CPU times: user 7.6 s, sys: 3.57 s, total: 11.2 s
Wall time: 35.5 s


In [24]:
dico['Perceptron'] = grid_per.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

#### Observation 

La regression logistique n'est qu'un perceptron avec une sigmoid en fonction d'activation.
On voit que la Regression Logistique a de meilleures performances à l'issue de la recherche d'hyperparamètres mais pas de loin. Par ailleurs le temps d'entrainement est considérablement plus élevé pour la regression logistique (du au calcul de l'exponentiel). Nous verrons par la suite quel modèle il est préférable de conserver. 

### LinearSVC

In [25]:
%%time
grid_svc = grid_Search("SVC") 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


/home/eliott/envs/RNenv/lib64/python3.10/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/eliott/envs/RNenv/lib64/python3.10/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/eliott/envs/RNenv/lib64/python3.10/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/eliott/envs/RNenv/lib64/python3.10/site-packages/sklearn/svm/_classes.py", line 257, in fit
    self.coef_, se

Best Score:  0.8689651633428769
Best Params:  {'clf__dual': False, 'clf__loss': 'squared_hinge', 'clf__penalty': 'l1'}
CPU times: user 6.25 s, sys: 102 ms, total: 6.35 s
Wall time: 21.9 s


In [26]:
dico['SVC'] = grid_svc.best_params_
with open('data.json', 'w') as fp:
    json.dump(dico, fp,  indent=4)

### Charging the best parameters for training

In [27]:
# Opening JSON file
with open('data.json') as json_file:
    data = json.load(json_file)


data_sgd = data['SGD']
data_rf = data['RF']
data_gb = data['GB']
data_lr = data['LR']
data_per = data['Perceptron']
data_svc = data['SVC']



### Training on all training data and testing on test data

In [28]:
def testing(model):
    predictions_train = model.predict(X_train)
    predictions_test = model.predict(X_test)
    accuracy_train = accuracy_score(y_train,predictions_train )
    accuracy_test = accuracy_score(y_test,predictions_test )
    
    print(f"train_accuracy : {accuracy_train} \ntest_accuracy : {accuracy_test}  ")




### SGD

In [29]:
model_sgd_best = pipeline_sgd
model_sgd_best.set_params(**data_sgd,clf__n_jobs=-1) 

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.0615789473684212e-05, n_jobs=-1,
                               penalty='l1'))])

In [30]:
%%time

"""
Si on a fait la recherche avant, on prend le best modèle,

Sinon, on reconstruit le modèle avec les best paramètres et on le fit à toutes les données d'entrainement. 

"""

model_sgd_best.fit(X_train,y_train)


CPU times: user 1.49 s, sys: 996 ms, total: 2.48 s
Wall time: 871 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 SGDClassifier(alpha=1.0615789473684212e-05, n_jobs=-1,
                               penalty='l1'))])

In [31]:
testing(model_sgd_best)

train_accuracy : 0.9414437398255461 
test_accuracy : 0.8841495523959979  


### Random Forest

In [32]:
model_rf_best = pipeline_rf
model_rf_best.set_params(**data_rf,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

In [33]:
%%time

model_rf_best.fit(X_train,y_train)

CPU times: user 21.3 s, sys: 84.9 ms, total: 21.4 s
Wall time: 2.07 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 RandomForestClassifier(max_depth=600, max_features='sqrt',
                                        min_samples_leaf=2,
                                        min_samples_split=32, n_jobs=-1))])

In [34]:
testing(model_rf_best)

train_accuracy : 0.8690137765143232 
test_accuracy : 0.6901000526592944  


### Gradient Boosting 

In [35]:
model_gb_best = pipeline_gb
model_gb_best.set_params(**data_gb)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [36]:
%%time
model_gb_best.fit(X_train,y_train)

CPU times: user 8min 56s, sys: 0 ns, total: 8min 56s
Wall time: 8min 57s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            n_estimators=800))])

In [37]:
testing(model_gb_best)

train_accuracy : 0.9167334839759944 
test_accuracy : 0.8307003686150606  


### Logistic Regression

In [38]:
model_lr_best = pipeline_lr
model_lr_best.set_params(**data_lr,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

In [39]:
%%time

model_lr_best.fit(X_train,y_train)

CPU times: user 610 ms, sys: 170 ms, total: 780 ms
Wall time: 10.3 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=10, max_iter=1000, n_jobs=-1,
                                    tol=1e-08))])

In [40]:
testing(model_lr_best)

train_accuracy : 0.9755327161843672 
test_accuracy : 0.833070036861506  


### Perceptron

In [41]:
model_per_best = pipeline_per
model_per_best.set_params(**data_per,clf__n_jobs=-1)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

In [42]:
%%time
model_per_best.fit(X_train,y_train)

CPU times: user 1.07 s, sys: 1.16 s, total: 2.23 s
Wall time: 727 ms


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf',
                 Perceptron(alpha=5.272631578947369e-06, n_jobs=-1,
                            penalty='l1'))])

In [43]:
testing(model_per_best)

train_accuracy : 0.9055324732123332 
test_accuracy : 0.822538177988415  


### SVC

In [44]:
model_svc_best = pipeline_svc
model_svc_best.set_params(**data_svc)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [45]:
%%time

model_svc_best.fit(X_train,y_train)

CPU times: user 4.68 s, sys: 8.3 ms, total: 4.68 s
Wall time: 4.69 s


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LinearSVC(dual=False, max_iter=10000, penalty='l1'))])

In [46]:
testing(model_svc_best)

train_accuracy : 0.9481983623684914 
test_accuracy : 0.8730911005792522  


### Voting Classifier

In [47]:
from sklearn.ensemble import VotingClassifier


In [48]:
clf1 = model_sgd_best

clf2 = model_rf_best

clf3 = model_gb_best

clf4 = model_lr_best

clf5 = model_per_best

clf6 = model_svc_best


eclf1 = VotingClassifier(
     estimators=[('sgd', clf1), ('rf', clf2), ('gb', clf3), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')

eclf2 = VotingClassifier(
     estimators=[('sgd', clf1), ('lr', clf4), ('per', clf5), ('svc', clf6)],
     voting='hard')


In [49]:
%%time

eclf1.fit(X_train,y_train)

CPU times: user 8min 53s, sys: 2.19 s, total: 8min 55s
Wall time: 9min 4s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('rf',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               RandomForestClassifier(max_depth=600,
                                                                      max_features='sqrt',
                                                                      min_samples_leaf=2,
                                                                      min_samples_split=32,
                                                                  

In [50]:
testing(eclf1)

train_accuracy : 0.9578686493184635 
test_accuracy : 0.8717746182201158  


In [51]:
%%time

eclf2.fit(X_train,y_train)

CPU times: user 7.5 s, sys: 2.08 s, total: 9.58 s
Wall time: 16.3 s


VotingClassifier(estimators=[('sgd',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               SGDClassifier(alpha=1.0615789473684212e-05,
                                                             n_jobs=-1,
                                                             penalty='l1'))])),
                             ('lr',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf',
                                               LogisticRegression(C=10,
                                                                  max_iter=1000,
                                                                  n_jobs=-1,
                                                                  tol=1e-08))])),
                             ('per',
                              Pipeline(steps=[('tfidf', TfidfVectorizer()),

In [52]:
testing(eclf2)

train_accuracy : 0.9575527856743689 
test_accuracy : 0.8688783570300158  


### Tests perso

In [53]:
tests_perso = ["I think covid is the most horrible threat we ever faced",
                "I love covid, thanks to it I can see my family much more often and I don't have to comute as much",
                "I would love to come to your birthday party, but I got covid, I have to stay confined"
]

In [54]:
predictions_out = eclf2.predict(tests_perso)


In [55]:
predictions_out

array([-1,  1,  1])